**Summary**<br>
This section provides the result summary of 3 different network architecture in order to save your time by running it separately.<br>
In the required baseline system,by modifying the code provided by professor,the accuracy after each epoch is showing below:<br>
Epoch 0: train set accuracy 0.6758166666666666<br>
Epoch 1: train set accuracy 0.74505<br>
Epoch 2: train set accuracy 0.77195<br>
Epoch 3: train set accuracy 0.78775<br>
Epoch 4: train set accuracy 0.80455<br>
Epoch 5: train set accuracy 0.8196333333333333<br>
Epoch 6: train set accuracy 0.8300333333333333<br>
Epoch 7: train set accuracy 0.8365166666666667<br>
Epoch 8: train set accuracy 0.8419333333333333<br>
Epoch 9: train set accuracy 0.84615<br>
Epoch 9: test set accuracy 0.8371<br>


For the fist comparision system, I choose to vary the activation functions in layer 4 where I change the ReLu to tanh. The performance is given below:<br>
Epoch 0: train set accuracy 0.6758166666666666<br>
Epoch 1: train set accuracy 0.74505<br>
Epoch 2: train set accuracy 0.77195<br>
Epoch 3: train set accuracy 0.78775<br>
Epoch 4: train set accuracy 0.80455<br>
Epoch 5: train set accuracy 0.8196333333333333<br>
Epoch 6: train set accuracy 0.8300333333333333<br>
Epoch 7: train set accuracy 0.8365166666666667<br>
Epoch 8: train set accuracy 0.8419333333333333<br>
Epoch 9: train set accuracy 0.84615<br>
Epoch 9: test set accuracy 0.8371<br>
As you can see, we almost get the same value compared with baseline system. I would state layer 4 does contribute too much in this setting since change the activation function almost result in no difference.

For the second comparison system, I choose to vary the number of channels in the first two layers. The result is provided below:<br>
Epoch 0: train set accuracy 0.6422<br>
Epoch 1: train set accuracy 0.7377166666666667<br>
Epoch 2: train set accuracy 0.7731833333333333<br>
Epoch 3: train set accuracy 0.7941333333333334<br>
Epoch 4: train set accuracy 0.8130333333333334<br>
Epoch 5: train set accuracy 0.82915<br>
Epoch 6: train set accuracy 0.84035<br>
Epoch 7: train set accuracy 0.8493333333333334<br>
Epoch 8: train set accuracy 0.8565666666666667<br>
Epoch 9: train set accuracy 0.86285<br>
Epoch 9: test set accuracy 0.8503<br>
As you can see, we get slight better result compared with the first two settings. This actually makes sense since we increase the output channels so that the network can capture more information in the image data. If we keep increase the channel, we might get a better accuracy. However, I did not choose to experiment on more channels since training takes too long.


**Baseline System**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter # TensorBoard support


# import torchvision module to handle image manipulation
import torchvision
import torchvision.transforms as transforms

In [2]:
# Use standard FashionMNIST dataset
train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)

test_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = False,
    download = False,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)


Extracting ./data/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw




Extracting ./data/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw




Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw




Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw

Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [5]:
class BaselineNet(nn.Module):

    def __init__(self):
        super(BaselineNet, self).__init__()

        
        # layer 1 
        self.layer_1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # layer 2 
        self.layer_2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # layer 3 
        self.layer_3 = nn.Sequential(
            nn.Linear(in_features=12 * 4 * 4, out_features=120),
            nn.ReLU()
        )

        # layer 4 
        self.layer_4 = nn.Sequential(
            nn.Linear(in_features=120, out_features=60),
            nn.ReLU()
        )

        # layer 5 
        self.layer_5 = nn.Sequential(
            nn.Linear(in_features=60, out_features=10)
        )


    def forward(self, feature):
        
        # reshape the feature
        t = feature.reshape(-1, 1, 28, 28)

        # forward the feature
        t = self.layer_1(feature)
        t = self.layer_2(t)

        # reshape the prediction
        t = t.reshape(-1, 12 * 4 * 4)

        # forward the feature through fully connected layers
        t = self.layer_3(t)
        t = self.layer_4(t)
        t = self.layer_5(t)

        return t
  
net = BaselineNet()

In [3]:
def get_accuracy(model,dataloader):
  count=0
  correct=0

  model.eval()
  with torch.no_grad():
    for batch in dataloader:
      images = batch[0]
      labels = batch[1]
      preds=network(images)
      batch_correct=preds.argmax(dim=1).eq(labels).sum().item()
      batch_count=len(batch[0])
      count+=batch_count
      correct+=batch_correct
  model.train()
  return correct/count

In [6]:
lr=0.001
batch_size=1000
shuffle=True
epochs=10

network = BaselineNet()
loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size)
optimizer = optim.Adam(network.parameters(), lr=lr)

# set the network to training mode
network.train()
for epoch in range(epochs):
  for batch in loader:
    images = batch[0]
    labels = batch[1]
    preds = network(images)
    loss = F.cross_entropy(preds, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print('Epoch {0}: train set accuracy {1}'.format(epoch,get_accuracy(network,loader)))

test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size)
print('Epoch {0}: test set accuracy {1}'.format(epoch,get_accuracy(network,test_loader)))

Epoch 0: train set accuracy 0.6758166666666666
Epoch 1: train set accuracy 0.74505
Epoch 2: train set accuracy 0.77195
Epoch 3: train set accuracy 0.78775
Epoch 4: train set accuracy 0.80455
Epoch 5: train set accuracy 0.8196333333333333
Epoch 6: train set accuracy 0.8300333333333333
Epoch 7: train set accuracy 0.8365166666666667
Epoch 8: train set accuracy 0.8419333333333333
Epoch 9: train set accuracy 0.84615
Epoch 9: test set accuracy 0.8371


**Comparision 1**

In [7]:
class NN1(nn.Module):

    def __init__(self):
        super(NN1, self).__init__()

        # layer 1 
        self.layer_1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # layer 2 
        self.layer_2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # layer 3 
        self.layer_3 = nn.Sequential(
            nn.Linear(in_features=12 * 4 * 4, out_features=120),
            nn.ReLU()
        )

        # layer 4 
        self.layer_4 = nn.Sequential(
            nn.Linear(in_features=120, out_features=60),
            nn.Tanh()
        )

        # layer 5 
        self.layer_5 = nn.Sequential(
            nn.Linear(in_features=60, out_features=10)
        )


    def forward(self, feature):
        
        # reshape the feature
        t = feature.reshape(-1, 1, 28, 28)

        # forward the feature
        t = self.layer_1(feature)
        t = self.layer_2(t)

        # reshape the prediction
        t = t.reshape(-1, 12 * 4 * 4)

        # forward the feature through fully connected layers
        t = self.layer_3(t)
        t = self.layer_4(t)
        t = self.layer_5(t)

        return t
  
net = NN1()

In [9]:
lr=0.001
batch_size=1000
shuffle=True
epochs=10

network = NN1()
loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size)
optimizer = optim.Adam(network.parameters(), lr=lr)

# set the network to training mode
network.train()
for epoch in range(epochs):
  for batch in loader:
    images = batch[0]
    labels = batch[1]
    preds = network(images)
    loss = F.cross_entropy(preds, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print('Epoch {0}: train set accuracy {1}'.format(epoch,get_accuracy(network,loader)))

test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size)
print('Epoch {0}: test set accuracy {1}'.format(epoch,get_accuracy(network,test_loader)))

Epoch 0: train set accuracy 0.6750166666666667
Epoch 1: train set accuracy 0.7395333333333334
Epoch 2: train set accuracy 0.7678
Epoch 3: train set accuracy 0.7830166666666667
Epoch 4: train set accuracy 0.7941333333333334
Epoch 5: train set accuracy 0.80405
Epoch 6: train set accuracy 0.8126666666666666
Epoch 7: train set accuracy 0.8205333333333333
Epoch 8: train set accuracy 0.828
Epoch 9: train set accuracy 0.8367166666666667
Epoch 9: test set accuracy 0.8245


In [10]:
class NN2(nn.Module):

    def __init__(self):
        super(NN2, self).__init__()

        # kernal
        # layer 1 
        self.layer_1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=12, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # layer 2 
        self.layer_2 = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # layer 3 
        self.layer_3 = nn.Sequential(
            nn.Linear(in_features=24 * 4 * 4, out_features=240),
            nn.ReLU()
        )

        # layer 4 
        self.layer_4 = nn.Sequential(
            nn.Linear(in_features=240, out_features=120),
            nn.ReLU()
        )

        # layer 5 
        self.layer_5 = nn.Sequential(
            nn.Linear(in_features=120, out_features=20)
        )


    def forward(self, feature):
        
        # reshape the feature
        t = feature.reshape(-1, 1, 28, 28)

        # forward the feature
        t = self.layer_1(feature)
        t = self.layer_2(t)

        # reshape the prediction 
        t = t.reshape(-1, 24 * 4 * 4)

        # forward the feature through fully connected layers
        t = self.layer_3(t)
        t = self.layer_4(t)
        t = self.layer_5(t)

        return t
  
net = NN2()

In [12]:
lr=0.001
batch_size=1000
shuffle=True
epochs=10

network = NN2()
loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size)
optimizer = optim.Adam(network.parameters(), lr=lr)

# set the network to training mode
network.train()
for epoch in range(epochs):
  for batch in loader:
    images = batch[0]
    labels = batch[1]
    preds = network(images)
    loss = F.cross_entropy(preds, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print('Epoch {0}: train set accuracy {1}'.format(epoch,get_accuracy(network,loader)))

test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size)
print('Epoch {0}: test set accuracy {1}'.format(epoch,get_accuracy(network,test_loader)))

Epoch 0: train set accuracy 0.6422
Epoch 1: train set accuracy 0.7377166666666667
Epoch 2: train set accuracy 0.7731833333333333
Epoch 3: train set accuracy 0.7941333333333334
Epoch 4: train set accuracy 0.8130333333333334
Epoch 5: train set accuracy 0.82915
Epoch 6: train set accuracy 0.84035
Epoch 7: train set accuracy 0.8493333333333334
Epoch 8: train set accuracy 0.8565666666666667
Epoch 9: train set accuracy 0.86285
Epoch 9: test set accuracy 0.8503
